<a href="https://colab.research.google.com/github/ThilinaRajapakse/pytorch-transformers-classification/blob/master/colab_quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/My Drive/BinaryClassification_COVID

/content/drive/My Drive/BinaryClassification_COVID


In [3]:
!pip install pytorch-transformers
#!pip install transformers

     |████████████████████████████████| 184kB 3.5MB/s 
     |████████████████████████████████| 890kB 10.5MB/s 
     |████████████████████████████████| 1.1MB 17.4MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=005199d77b80bb03b2294a183706c02c70f303848428d6faf9e68ba14e0b4007
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [4]:
%%writefile utils.py 

# coding=utf-8
# Copyright 2018 The Google AI Language Team Authors and The HuggingFace Inc. team.
# Copyright (c) 2018, NVIDIA CORPORATION.  All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
""" BERT classification fine-tuning: utilities to work with GLUE tasks """

from __future__ import absolute_import, division, print_function

import csv
import logging
import os
import sys
from io import open

from scipy.stats import pearsonr, spearmanr
from sklearn.metrics import matthews_corrcoef, f1_score

from multiprocessing import Pool, cpu_count
from tqdm import tqdm

logger = logging.getLogger(__name__)
csv.field_size_limit(2147483647)

class InputExample(object):
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None, label=None):
        """Constructs a InputExample.

        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence. For single
            sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second sequence.
            Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
            specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self, input_ids, input_mask, segment_ids, label_id):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id


class DataProcessor(object):
    """Base class for data converters for sequence classification data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """Reads a tab separated value file."""
        with open(input_file, "r", encoding="utf-8-sig") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                if sys.version_info[0] == 2:
                    line = list(unicode(cell, 'utf-8') for cell in line)
                lines.append(line)
            return lines


class BinaryProcessor(DataProcessor):
    """Processor for the binary data sets"""

    def get_train_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train_subset5.tsv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "test_set.tsv")), "dev")

    def get_labels(self):
        """See base class."""
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        """Creates examples for the training and dev sets."""
        examples = []
        for (i, line) in enumerate(lines):
            guid = "%s-%s" % (set_type, i)
            text_a = line[3]
            label = line[1]
            examples.append(
                InputExample(guid=guid, text_a=text_a, text_b=None, label=label))
        return examples


def convert_example_to_feature(example_row, pad_token=0,
sequence_a_segment_id=0, sequence_b_segment_id=1,
cls_token_segment_id=1, pad_token_segment_id=0,
mask_padding_with_zero=True):
    example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id = example_row

    tokens_a = tokenizer.tokenize(example.text_a)

    tokens_b = None
    if example.text_b:
        tokens_b = tokenizer.tokenize(example.text_b)
        # Modifies `tokens_a` and `tokens_b` in place so that the total
        # length is less than the specified length.
        # Account for [CLS], [SEP], [SEP] with "- 3"
        _truncate_seq_pair(tokens_a, tokens_b, max_seq_length - 3)
    else:
        # Account for [CLS] and [SEP] with "- 2"
        if len(tokens_a) > max_seq_length - 2:
            tokens_a = tokens_a[:(max_seq_length - 2)]

    # The convention in BERT is:
    # (a) For sequence pairs:
    #  tokens:   [CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]
    #  type_ids:   0   0  0    0    0     0       0   0   1  1  1  1   1   1
    # (b) For single sequences:
    #  tokens:   [CLS] the dog is hairy . [SEP]
    #  type_ids:   0   0   0   0  0     0   0
    #
    # Where "type_ids" are used to indicate whether this is the first
    # sequence or the second sequence. The embedding vectors for `type=0` and
    # `type=1` were learned during pre-training and are added to the wordpiece
    # embedding vector (and position vector). This is not *strictly* necessary
    # since the [SEP] token unambiguously separates the sequences, but it makes
    # it easier for the model to learn the concept of sequences.
    #
    # For classification tasks, the first vector (corresponding to [CLS]) is
    # used as as the "sentence vector". Note that this only makes sense because
    # the entire model is fine-tuned.
    tokens = tokens_a + [sep_token]
    segment_ids = [sequence_a_segment_id] * len(tokens)

    if tokens_b:
        tokens += tokens_b + [sep_token]
        segment_ids += [sequence_b_segment_id] * (len(tokens_b) + 1)

    if cls_token_at_end:
        tokens = tokens + [cls_token]
        segment_ids = segment_ids + [cls_token_segment_id]
    else:
        tokens = [cls_token] + tokens
        segment_ids = [cls_token_segment_id] + segment_ids

    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # The mask has 1 for real tokens and 0 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [1 if mask_padding_with_zero else 0] * len(input_ids)

    # Zero-pad up to the sequence length.
    padding_length = max_seq_length - len(input_ids)
    if pad_on_left:
        input_ids = ([pad_token] * padding_length) + input_ids
        input_mask = ([0 if mask_padding_with_zero else 1] * padding_length) + input_mask
        segment_ids = ([pad_token_segment_id] * padding_length) + segment_ids
    else:
        input_ids = input_ids + ([pad_token] * padding_length)
        input_mask = input_mask + ([0 if mask_padding_with_zero else 1] * padding_length)
        segment_ids = segment_ids + ([pad_token_segment_id] * padding_length)

    assert len(input_ids) == max_seq_length
    assert len(input_mask) == max_seq_length
    assert len(segment_ids) == max_seq_length

    if output_mode == "classification":
        label_id = label_map[example.label]
    elif output_mode == "regression":
        label_id = float(example.label)
    else:
        raise KeyError(output_mode)

    return InputFeatures(input_ids=input_ids,
                        input_mask=input_mask,
                        segment_ids=segment_ids,
                        label_id=label_id)
    

def convert_examples_to_features(examples, label_list, max_seq_length,
                                 tokenizer, output_mode,
                                 cls_token_at_end=False, pad_on_left=False,
                                 cls_token='[CLS]', sep_token='[SEP]', pad_token=0,
                                 sequence_a_segment_id=0, sequence_b_segment_id=1,
                                 cls_token_segment_id=1, pad_token_segment_id=0,
                                 mask_padding_with_zero=True,
                                 process_count=cpu_count() - 2):
    """ Loads a data file into a list of `InputBatch`s
        `cls_token_at_end` define the location of the CLS token:
            - False (Default, BERT/XLM pattern): [CLS] + A + [SEP] + B + [SEP]
            - True (XLNet/GPT pattern): A + [SEP] + B + [SEP] + [CLS]
        `cls_token_segment_id` define the segment id associated to the CLS token (0 for BERT, 2 for XLNet)
    """

    label_map = {label : i for i, label in enumerate(label_list)}

    examples = [(example, label_map, max_seq_length, tokenizer, output_mode, cls_token_at_end, cls_token, sep_token, cls_token_segment_id, pad_on_left, pad_token_segment_id) for example in examples]

    with Pool(process_count) as p:
        features = list(tqdm(p.imap(convert_example_to_feature, examples, chunksize=100), total=len(examples)))

    return features


def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()


processors = {
    "binary": BinaryProcessor
}

output_modes = {
    "binary": "classification"
}

GLUE_TASKS_NUM_LABELS = {
    "binary": 2
}


Overwriting utils.py


In [5]:
from __future__ import absolute_import, division, print_function

import glob
import logging
import os
import random
import json

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
import random
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm_notebook, trange


from pytorch_transformers import (WEIGHTS_NAME, BertConfig, BertForSequenceClassification, BertTokenizer,
                                  XLMConfig, XLMForSequenceClassification, XLMTokenizer, 
                                  XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer,
                                  RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer,)

#from transformers import BertForPreTraining, AutoModel, AutoTokenizer

from pytorch_transformers import AdamW, WarmupLinearSchedule

from utils import (convert_examples_to_features,
                        output_modes, processors)

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

In [6]:
args = {
    'data_dir': 'data/bagging',
    # 'model_type':  'roberta',
    # 'model_name': 'roberta-base',
    'model_type':  'bert',
    'model_name': 'bert-large-uncased',
    'task_name': 'binary',
    'output_dir': 'outputs/',
    'cache_dir': 'cache/',
    'do_train': True,
    'do_eval': True,
    'fp16': False,
    'fp16_opt_level': 'O1',
    'max_seq_length': 128,
    'output_mode': 'classification',
    'train_batch_size': 8,
    'eval_batch_size': 8,

    'gradient_accumulation_steps': 1,
    'num_train_epochs': 10,
    'weight_decay': 0,
    'learning_rate': 4e-5,
    'adam_epsilon': 1e-8,
    'warmup_steps': 0,
    'max_grad_norm': 1.0,

    'logging_steps': 50,
    'evaluate_during_training': True,
    'save_steps': 2000,
    'eval_all_checkpoints': True,

    'overwrite_output_dir': False,
    'reprocess_input_data': False,
    'notes': 'Using Yelp Reviews dataset'
}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
args

{'adam_epsilon': 1e-08,
 'cache_dir': 'cache/',
 'data_dir': 'data/',
 'do_eval': True,
 'do_train': True,
 'eval_all_checkpoints': True,
 'eval_batch_size': 8,
 'evaluate_during_training': True,
 'fp16': False,
 'fp16_opt_level': 'O1',
 'gradient_accumulation_steps': 1,
 'learning_rate': 4e-05,
 'logging_steps': 50,
 'max_grad_norm': 1.0,
 'max_seq_length': 128,
 'model_name': 'distilbert-base-uncased',
 'model_type': 'distilbert',
 'notes': 'Using Yelp Reviews dataset',
 'num_train_epochs': 10,
 'output_dir': 'outputs/',
 'output_mode': 'classification',
 'overwrite_output_dir': False,
 'reprocess_input_data': False,
 'save_steps': 2000,
 'task_name': 'binary',
 'train_batch_size': 8,
 'warmup_steps': 0,
 'weight_decay': 0}

In [ ]:
with open('args.json', 'w') as f:
    json.dump(args, f)

In [ ]:
if os.path.exists(args['output_dir']) and os.listdir(args['output_dir']) and args['do_train'] and not args['overwrite_output_dir']:
    raise ValueError("Output directory ({}) already exists and is not empty. Use --overwrite_output_dir to overcome.".format(args['output_dir']))

In [7]:
!pip install transformers

     |████████████████████████████████| 778kB 3.2MB/s 
     |████████████████████████████████| 3.0MB 9.3MB/s 


In [8]:
#from transformers import BertConfig, BertForSequenceClassification, BertTokenizer
MODEL_CLASSES = {
    'bert': (BertConfig, BertForSequenceClassification, BertTokenizer),
    'xlnet': (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer),
    'xlm': (XLMConfig, XLMForSequenceClassification, XLMTokenizer),
    'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer),
    # 'albert': (AlbertConfig, AlbertForSequenceClassification ,AlbertTokenizer)
}

config_class, model_class, tokenizer_class = MODEL_CLASSES[args['model_type']]

In [ ]:
config = config_class.from_pretrained(args['model_name'], num_labels=2, finetuning_task=args['task_name'])
tokenizer = tokenizer_class.from_pretrained(args['model_name'])

INFO:filelock:Lock 140046643918216 acquired on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpfdegdcvm


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json in cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
INFO:filelock:Lock 140046643918216 released on /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690.lock
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json in cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b
INFO:filelock:Lock 140046643919672 released on /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock
INFO:filelock:Lock 140046643920288 acquired on /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggin

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-merges.txt in cache at /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda
INFO:filelock:Lock 140046643920288 released on /root/.cache/torch/transformers/b35e7cd126cd4229a746b5d5c29a749e8e84438b14bcdb575950584fe33207e8.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-vocab.json from cache at /root/.cache/torch/transformers/d0c5776499adc1ded22493fae699da0971c1ee4c2587111707a4d177d20257a2.ef00af9e673c7160b4d41cfda1f48c5f4cba57d

In [ ]:
model = model_class.from_pretrained(args['model_name'])

INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/roberta-base-config.json from cache at /root/.cache/torch/transformers/e1a2a406b5a05063c31f4dfdee7608986ba7c6393f7f79db5e69dcd197208534.117c81977c5979de8c088352e74ec6e70f5c66096c28b61d3c50101609b39690
INFO:transformers.configuration_utils:Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "type_vocab_size": 1,
  "vocab_size": 50265
}

INFO:filelock:Lock 140046643917936 acquired on /root/.cache/torch/transformers/80b4a484eddeb259bec2f

INFO:transformers.file_utils:storing https://cdn.huggingface.co/roberta-base-pytorch_model.bin in cache at /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e
INFO:filelock:Lock 140046643917936 released on /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e.lock
INFO:transformers.modeling_utils:loading weights file https://cdn.huggingface.co/roberta-base-pytorch_model.bin from cache at /root/.cache/torch/transformers/80b4a484eddeb259bec2f06a6f2f05d90934111628e0e1c09a33bd4a121358e1.49b88ba7ec2c26a7558dda98ca3884c3b80fa31cf43a1b1f23aef3ff81ba344e


- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import AutoTokenizer
config = config_class.from_pretrained('ctbert/config.json')
#tokenizer = tokenizer_class.from_pretrained('ctbert/')
tokenizer = AutoTokenizer.from_pretrained("digitalepidemiologylab/covid-twitter-bert")

INFO:transformers.file_utils:PyTorch version 1.6.0+cu101 available.
INFO:transformers.file_utils:TensorFlow version 2.3.0 available.
INFO:pytorch_transformers.modeling_utils:loading configuration file ctbert/config.json
INFO:pytorch_transformers.modeling_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pad_token_id": 0,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:filelock:Lock 140260746804584 acquired on /root/.cache/torch/transformers/7fe0c0c31e05dc4ee4352acd3d2f55769729af6112ef726b17d110728bd89b31.94acc

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/config.json in cache at /root/.cache/torch/transformers/7fe0c0c31e05dc4ee4352acd3d2f55769729af6112ef726b17d110728bd89b31.94acc9b2980ce55713e34d393e99698aa32bc14cfb1143c33dd4a0605b374b89
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/7fe0c0c31e05dc4ee4352acd3d2f55769729af6112ef726b17d110728bd89b31.94acc9b2980ce55713e34d393e99698aa32bc14cfb1143c33dd4a0605b374b89
INFO:filelock:Lock 140260746804584 released on /root/.cache/torch/transformers/7fe0c0c31e05dc4ee4352acd3d2f55769729af6112ef726b17d110728bd89b31.94acc9b2980ce55713e34d393e99698aa32bc14cfb1143c33dd4a0605b374b89.lock
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/config.json from cache at /root/.cache/torch/transformers/7fe0c0c31e05dc4ee4352acd3d2f557697

INFO:filelock:Lock 140261007103200 acquired on /root/.cache/torch/transformers/b042b84bc66b83a767b8b3bb612da63e36440e54a3fee7f7e3dba9a90b3ff919.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
INFO:transformers.file_utils:https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/vocab.txt not found in cache or force_download set to True, downloading to /root/.cache/torch/transformers/tmpl049ytbw


INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/vocab.txt in cache at /root/.cache/torch/transformers/b042b84bc66b83a767b8b3bb612da63e36440e54a3fee7f7e3dba9a90b3ff919.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/b042b84bc66b83a767b8b3bb612da63e36440e54a3fee7f7e3dba9a90b3ff919.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:filelock:Lock 140261007103200 released on /root/.cache/torch/transformers/b042b84bc66b83a767b8b3bb612da63e36440e54a3fee7f7e3dba9a90b3ff919.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/vocab.txt from cache at /root/.cache/torch/transformers/b042b84bc66b83a767b8b3bb612da63e36440e54a3fee7f7e3dba9a90b3ff919.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/added_tokens.json from cache at None
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/special_tokens_map.json from cache at None
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/tokenizer_config.json from cache at None
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingfac

In [ ]:
model = model_class.from_pretrained('ctbert/')

INFO:pytorch_transformers.modeling_utils:loading configuration file ctbert/config.json
INFO:pytorch_transformers.modeling_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pad_token_id": 0,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:pytorch_transformers.modeling_utils:loading weights file ctbert/pytorch_model.bin
INFO:pytorch_transformers.modeling_utils:Weights of BertForSequenceClassification not initialized from pretrained model: ['classifier.weight', 'classifier.bias']
INFO:pytorch_transformers.modelin

In [ ]:
model.to(device);

In [ ]:
device

device(type='cuda')

In [10]:
task = args['task_name']

processor = processors[task]()
label_list = processor.get_labels()
num_labels = len(label_list)

In [11]:
def load_and_cache_examples(task, tokenizer, evaluate=False, undersample_scale_factor=1):
    processor = processors[task]()
    output_mode = args['output_mode']
    
    mode = 'dev' if evaluate else 'train'
    cached_features_file = os.path.join(args['data_dir'], f"cached4_{mode}_{args['model_name']}_{args['max_seq_length']}_{task}")
    
   # if os.path.exists(cached_features_file) and not args['reprocess_input_data']:
   #    logger.info("Loading features from cached file %s", cached_features_file)
   #     features = torch.load(cached_features_file)
               
   # else:
        
    logger.info("Creating features from dataset file at %s", args['data_dir'])
    label_list = processor.get_labels()
    print("Hi")
    examples = processor.get_dev_examples(args['data_dir']) if evaluate else processor.get_train_examples(args['data_dir'])
    print(len(examples))
    examples  = [example for example in examples if np.random.rand() < undersample_scale_factor]
    print(len(examples))
        
    features = convert_examples_to_features(examples, label_list, args['max_seq_length'], tokenizer, output_mode,
        cls_token_at_end=bool(args['model_type'] in ['xlnet']),            # xlnet has a cls token at the end
        cls_token=tokenizer.cls_token,
        sep_token=tokenizer.sep_token,
        cls_token_segment_id=2 if args['model_type'] in ['xlnet'] else 0,
        pad_on_left=bool(args['model_type'] in ['xlnet']),                 # pad on the left for xlnet
        pad_token_segment_id=4 if args['model_type'] in ['xlnet'] else 0,
        process_count=2)
        
    logger.info("Saving features into cached file %s", cached_features_file)
    torch.save(features, cached_features_file)
       
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    elif output_mode == "regression":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.float)

    dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    return dataset

In [12]:
def train(train_dataset, model, tokenizer):
    train_sampler = RandomSampler(train_dataset)
    train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=args['train_batch_size'])
    
    t_total = len(train_dataloader) // args['gradient_accumulation_steps'] * args['num_train_epochs']
    
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args['weight_decay']},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args['learning_rate'], eps=args['adam_epsilon'])
    scheduler = WarmupLinearSchedule(optimizer, warmup_steps=args['warmup_steps'], t_total=t_total)
    
    if args['fp16']:
        try:
            from apex import amp
        except ImportError:
            raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
        model, optimizer = amp.initialize(model, optimizer, opt_level=args['fp16_opt_level'])
        
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args['num_train_epochs'])
    logger.info("  Total train batch size  = %d", args['train_batch_size'])
    logger.info("  Gradient Accumulation steps = %d", args['gradient_accumulation_steps'])
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    tr_loss, logging_loss = 0.0, 0.0
    model.zero_grad()
    train_iterator = trange(int(args['num_train_epochs']), desc="Epoch")
    
    for epoch_no in train_iterator:
        epoch_iterator = tqdm_notebook(train_dataloader, desc="Iteration")
        for step, batch in enumerate(epoch_iterator):
            model.train()
            batch = tuple(t.to(device) for t in batch)
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
            outputs = model(**inputs)
            #print(outputs)
            loss = outputs[0]  # model outputs are always tuple in pytorch-transformers (see doc)
            
            print("\r%f" % loss, end='')

            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']

            if args['fp16']:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args['max_grad_norm'])
                
            else:
                loss.backward()
                torch.nn.utils.clip_grad_norm_(model.parameters(), args['max_grad_norm'])

            tr_loss += loss.item()
            if (step + 1) % args['gradient_accumulation_steps'] == 0:
                scheduler.step()  # Update learning rate schedule
                optimizer.step()
                model.zero_grad()
                global_step += 1

                # if args['logging_steps'] > 0 and global_step % args['logging_steps'] == 0:
                #     # Log metrics
                #     if args['evaluate_during_training']:  # Only evaluate when single GPU otherwise metrics may not average well
                #         results = evaluate(model, tokenizer, epoch_no)

                    # logging_loss = tr_loss

                if args['save_steps'] > 0 and global_step % args['save_steps'] == 0:
                    # Save model checkpoint
                    output_dir = os.path.join(args['output_dir'], 'checkpoint-{}'.format(global_step))
                    if not os.path.exists(output_dir):
                        os.makedirs(output_dir)
                    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    logger.info("Saving model checkpoint to %s", output_dir)

        if args['evaluate_during_training']:  # Only evaluate when single GPU otherwise metrics may not average well
            results = evaluate(model, tokenizer, str(epoch_no))    


    return global_step, tr_loss / global_step

In [13]:
from sklearn.metrics import mean_squared_error, matthews_corrcoef, confusion_matrix
from scipy.stats import pearsonr

def get_mismatched(labels, preds):
    mismatched = labels != preds
    examples = processor.get_dev_examples(args['data_dir'])
    wrong = [i for (i, v) in zip(examples, mismatched) if v]
    
    return wrong

def get_eval_report(labels, preds):
    mcc = matthews_corrcoef(labels, preds)
    tn, fp, fn, tp = confusion_matrix(labels, preds).ravel()
    precision = tp/(tp+fp) 
    recall = tp/(tp+fn) 
    fscore = (2*precision*recall)/(precision + recall)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return {
        "mcc": mcc,
        "tp": tp,
        "tn": tn,
        "fp": fp,
        "fn": fn,
        "fscore": fscore,
        "accuracy": accuracy
    }, get_mismatched(labels, preds)

def compute_metrics(task_name, preds, labels, epoch_no):
    assert len(preds) == len(labels)
    filename = 'final_predictions/final_predictions_'+ args['model_type'] +'' + epoch_no +'epochs_test_set.txt'
    with open(filename,'w') as writer:
        for pred in preds:
          if pred>=0.5:
            writer.write('INFORMATIVE')
          else:
            writer.write('UNINFORMATIVE')
          
          writer.write('\n')

    return get_eval_report(labels, preds)

def evaluate(model, tokenizer, epoch_no, prefix="" ):
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args['output_dir']

    results = {}
    EVAL_TASK = args['task_name']

    eval_dataset = load_and_cache_examples(EVAL_TASK, tokenizer, evaluate=True)
    if not os.path.exists(eval_output_dir):
        os.makedirs(eval_output_dir)


    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'])

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    eval_loss = 0.0
    nb_eval_steps = 0
    preds = None
    out_label_ids = None
    for batch in tqdm_notebook(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)

        with torch.no_grad():
            inputs = {'input_ids':      batch[0],
                      'attention_mask': batch[1],
                      'token_type_ids': batch[2] if args['model_type'] in ['bert', 'xlnet'] else None,  # XLM don't use segment_ids
                      'labels':         batch[3]}
            outputs = model(**inputs)
            tmp_eval_loss, logits = outputs[:2]

            eval_loss += tmp_eval_loss.mean().item()
        nb_eval_steps += 1
        if preds is None:
            preds = logits.detach().cpu().numpy()
            out_label_ids = inputs['labels'].detach().cpu().numpy()
        else:
            preds = np.append(preds, logits.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, inputs['labels'].detach().cpu().numpy(), axis=0)

    eval_loss = eval_loss / nb_eval_steps
    if args['output_mode'] == "classification":
        preds = np.argmax(preds, axis=1)
    elif args['output_mode'] == "regression":
        preds = np.squeeze(preds)
    result, wrong = compute_metrics(EVAL_TASK, preds, out_label_ids, epoch_no)
    results.update(result)

    output_eval_file = os.path.join(eval_output_dir, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return results, wrong

In [ ]:
# IMPORTANT #
# Due to the 12 hour limit on Google Colab and the time it would take to convert the dataset into features, the load_and_cache_examples() function has been modified
# to randomly undersample the dataset by a scale of 0.1

if args['do_train']:
    train_dataset = load_and_cache_examples(task, tokenizer)
    global_step, tr_loss = train(train_dataset, model, tokenizer)
    logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

INFO:__main__:Creating features from dataset file at data/bagging


Hi
7000
7000


100%|██████████| 7000/7000 [00:05<00:00, 1321.83it/s]
INFO:__main__:Saving features into cached file data/bagging/cached4_train_bert-large-uncased_128_binary
INFO:__main__:***** Running training *****
INFO:__main__:  Num examples = 7000
INFO:__main__:  Num Epochs = 10
INFO:__main__:  Total train batch size  = 8
INFO:__main__:  Gradient Accumulation steps = 1
INFO:__main__:  Total optimization steps = 8750
Epoch:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


1.009434

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/pytorch_transformers/optimization.py:166: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1).add_(1.0 - beta1, grad)


...

0.000109

INFO:__main__:Creating features from dataset file at data/bagging



Hi
1000
1000



100%|██████████| 1000/1000 [00:00<00:00, 1447.04it/s]
INFO:__main__:Saving features into cached file data/bagging/cached4_dev_bert-large-uncased_128_binary
INFO:__main__:***** Running evaluation  *****
INFO:__main__:  Num examples = 1000
INFO:__main__:  Batch size = 8


INFO:__main__:***** Eval results  *****
INFO:__main__:  accuracy = 0.959
INFO:__main__:  fn = 21
INFO:__main__:  fp = 20
INFO:__main__:  fscore = 0.9563365282215123
INFO:__main__:  mcc = 0.9176956701392639
INFO:__main__:  tn = 510
INFO:__main__:  tp = 449
Epoch: 100%|██████████| 10/10 [1:57:28<00:00, 704.80s/it]
INFO:__main__: global_step = 8750, average loss = 0.08879762603413213


In [15]:
#SUBSET5 
from transformers import AutoTokenizer
model = model_class.from_pretrained('outputs/checkpoint-4000/')
model.to(device)
tokenizer = AutoTokenizer.from_pretrained("digitalepidemiologylab/covid-twitter-bert")
result, wrong_preds = evaluate(model, tokenizer, epoch_no ="test_subset5")
        

INFO:pytorch_transformers.modeling_utils:loading configuration file outputs/checkpoint-4000/config.json
INFO:pytorch_transformers.modeling_utils:Model config {
  "attention_probs_dropout_prob": 0.1,
  "finetuning_task": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "num_labels": 2,
  "output_attentions": false,
  "output_hidden_states": false,
  "pad_token_id": 0,
  "pruned_heads": {},
  "torchscript": false,
  "type_vocab_size": 2,
  "vocab_size": 30522
}

INFO:pytorch_transformers.modeling_utils:loading weights file outputs/checkpoint-4000/pytorch_model.bin
INFO:filelock:Lock 139695800330560 acquired on /root/.cache/torch/transformers/7fe0c0c31e05dc4ee4352acd3d2f55769729af6112ef726b17d110728bd89b31.94acc9b2980ce5571

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/config.json in cache at /root/.cache/torch/transformers/7fe0c0c31e05dc4ee4352acd3d2f55769729af6112ef726b17d110728bd89b31.94acc9b2980ce55713e34d393e99698aa32bc14cfb1143c33dd4a0605b374b89
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/7fe0c0c31e05dc4ee4352acd3d2f55769729af6112ef726b17d110728bd89b31.94acc9b2980ce55713e34d393e99698aa32bc14cfb1143c33dd4a0605b374b89
INFO:filelock:Lock 139695800330560 released on /root/.cache/torch/transformers/7fe0c0c31e05dc4ee4352acd3d2f55769729af6112ef726b17d110728bd89b31.94acc9b2980ce55713e34d393e99698aa32bc14cfb1143c33dd4a0605b374b89.lock
INFO:transformers.configuration_utils:loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/config.json from cache at /root/.cache/torch/transformers/7fe0c0c31e05dc4ee4352acd3d2f557697

INFO:transformers.file_utils:storing https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/vocab.txt in cache at /root/.cache/torch/transformers/b042b84bc66b83a767b8b3bb612da63e36440e54a3fee7f7e3dba9a90b3ff919.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.file_utils:creating metadata file for /root/.cache/torch/transformers/b042b84bc66b83a767b8b3bb612da63e36440e54a3fee7f7e3dba9a90b3ff919.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:filelock:Lock 139697724892608 released on /root/.cache/torch/transformers/b042b84bc66b83a767b8b3bb612da63e36440e54a3fee7f7e3dba9a90b3ff919.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock


INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/vocab.txt from cache at /root/.cache/torch/transformers/b042b84bc66b83a767b8b3bb612da63e36440e54a3fee7f7e3dba9a90b3ff919.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/added_tokens.json from cache at None
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/special_tokens_map.json from cache at None
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingface.co/bert/digitalepidemiologylab/covid-twitter-bert/tokenizer_config.json from cache at None
INFO:transformers.tokenization_utils_base:loading file https://s3.amazonaws.com/models.huggingfac

Hi
12000
12000


100%|██████████| 12000/12000 [00:09<00:00, 1253.44it/s]
INFO:__main__:Saving features into cached file data/bagging/cached4_dev_bert-large-uncased_128_binary
INFO:__main__:***** Running evaluation  *****
INFO:__main__:  Num examples = 12000
INFO:__main__:  Batch size = 8
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:900: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)
INFO:__main__:***** Eval results  *****
INFO:__main__:  accuracy = 0.64175
INFO:__main__:  fn = 4299
INFO:__main__:  fp = 0
INFO:__main__:  fscore = 0.7817877265113446
INFO:__main__:  mcc = 0.0
INFO:__main__:  tn = 0
INFO:__main__:  tp = 7701


In [ ]:
if args['do_train']:
    if not os.path.exists(args['output_dir']):
            os.makedirs(args['output_dir'])
    logger.info("Saving model checkpoint to %s", args['output_dir'])
    
    model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
    model_to_save.save_pretrained(args['output_dir'])
    tokenizer.save_pretrained(args['output_dir'])
    torch.save(args, os.path.join(args['output_dir'], 'training_args.bin'))


INFO:__main__:Saving model checkpoint to outputs/


In [ ]:
results = {}
if args['do_eval']:
    checkpoints = [args['output_dir']]
    if args['eval_all_checkpoints']:
        checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(args['output_dir'] + '/**/' + WEIGHTS_NAME, recursive=True)))
        logging.getLogger("pytorch_transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
    logger.info("Evaluate the following checkpoints: %s", checkpoints)
    for checkpoint in checkpoints:
        global_step = checkpoint.split('-')[-1] if len(checkpoints) > 1 else ""
        model = model_class.from_pretrained(checkpoint)
        model.to(device)
        result, wrong_preds = evaluate(model, tokenizer, epoch_no = str(args["num_train_epochs"]), prefix=global_step)
        result = dict((k + '_{}'.format(global_step), v) for k, v in result.items())
        results.update(result)


INFO:__main__:Evaluate the following checkpoints: ['outputs/checkpoint-2000', 'outputs/checkpoint-4000', 'outputs']
INFO:__main__:Creating features from dataset file at data/


Hi
1000
1000










  0%|          | 0/1000 [00:00<?, ?it/s]







  0%|          | 1/1000 [00:00<07:02,  2.37it/s]







 10%|█         | 101/1000 [00:00<04:26,  3.37it/s]







 20%|██        | 201/1000 [00:00<02:46,  4.80it/s]







 30%|███       | 301/1000 [00:01<01:42,  6.83it/s]







 40%|████      | 401/1000 [00:01<01:01,  9.71it/s]







 50%|█████     | 501/1000 [00:01<00:36, 13.76it/s]







 60%|██████    | 601/1000 [00:01<00:20, 19.42it/s]







 70%|███████   | 701/1000 [00:01<00:10, 27.33it/s]







100%|██████████| 1000/1000 [00:02<00:00, 477.87it/s]
INFO:__main__:Saving features into cached file data/cached4_dev_roberta-base_128_binary
INFO:__main__:***** Running evaluation 2000 *****
INFO:__main__:  Num examples = 1000
INFO:__main__:  Batch size = 8
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:65: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


INFO:__main__:***** Eval results 2000 *****
INFO:__main__:  accuracy = 0.895
INFO:__main__:  fn = 34
INFO:__main__:  fp = 71
INFO:__main__:  fscore = 0.8929663608562691
INFO:__main__:  mcc = 0.7923796012701352
INFO:__main__:  tn = 457
INFO:__main__:  tp = 438


INFO:__main__:Creating features from dataset file at data/


...

INFO:__main__:***** Eval results 4000 *****
INFO:__main__:  accuracy = 0.903
INFO:__main__:  fn = 27
INFO:__main__:  fp = 70
INFO:__main__:  fscore = 0.9017223910840932
INFO:__main__:  mcc = 0.8093137018635782
INFO:__main__:  tn = 458
INFO:__main__:  tp = 445


INFO:__main__:Creating features from dataset file at data/


Hi
1000
1000










  0%|          | 0/1000 [00:00<?, ?it/s]







  0%|          | 1/1000 [00:00<06:11,  2.69it/s]







 10%|█         | 101/1000 [00:00<03:54,  3.83it/s]







 20%|██        | 201/1000 [00:00<02:26,  5.46it/s]







 30%|███       | 301/1000 [00:00<01:30,  7.76it/s]







 40%|████      | 401/1000 [00:01<00:54, 11.03it/s]







 50%|█████     | 501/1000 [00:01<00:32, 15.52it/s]







 70%|███████   | 701/1000 [00:01<00:13, 21.93it/s]







100%|██████████| 1000/1000 [00:01<00:00, 516.37it/s]
INFO:__main__:Saving features into cached file data/cached4_dev_roberta-base_128_binary
INFO:__main__:***** Running evaluation outputs *****
INFO:__main__:  Num examples = 1000
INFO:__main__:  Batch size = 8


INFO:__main__:***** Eval results outputs *****
INFO:__main__:  accuracy = 0.899
INFO:__main__:  fn = 39
INFO:__main__:  fp = 62
INFO:__main__:  fscore = 0.8955532574974147
INFO:__main__:  mcc = 0.7987332721964812
INFO:__main__:  tn = 466
INFO:__main__:  tp = 433


In [ ]:
results

{'fn_': 26,
 'fp_': 90,
 'fscore_': 0.884920634920635,
 'mcc_': 0.7752574792699938,
 'tn_': 438,
 'tp_': 446}